In [1]:
!git clone https://github.com/nateraw/huggingface-vit-finetune.git
%cd huggingface-vit-finetune/
!pip install -r requirements.txt
!pip install git+https://github.com/huggingface/transformers.git@master --upgrade
!pip install torchattacks

Cloning into 'huggingface-vit-finetune'...
remote: Enumerating objects: 24, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 24 (delta 10), reused 19 (delta 6), pack-reused 0
Unpacking objects: 100% (24/24), done.
/content/huggingface-vit-finetune
     |████████████████████████████████| 849kB 20.3MB/s 
     |████████████████████████████████| 276kB 32.2MB/s 
     |████████████████████████████████| 829kB 45.3MB/s 
     |████████████████████████████████| 112kB 54.5MB/s 
     |████████████████████████████████| 184kB 50.8MB/s 
     |████████████████████████████████| 1.3MB 52.0MB/s 
     |████████████████████████████████| 143kB 55.1MB/s 
     |████████████████████████████████| 296kB 55.5MB/s 
  Created wheel for PyYAML: filename=PyYAML-5.3.1-cp37-cp37m-linux_x86_64.whl size=44620 sha256=6e74ab124ead3d56270c6302c45affc2c8376e022adddb528a8cd831bfe81fa3
  Stored in directory: /root/.cache/pip/wheels/a7/c1/ea/cf5bd31012e735dc1dfea

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader

import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets

import matplotlib
import matplotlib.pyplot as plt
import numpy as np

from transformers import ViTFeatureExtractor, ViTForImageClassification
import requests

import torchattacks
import time
import json

In [3]:
result_dict = {}
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [4]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [6]:
# TODO: Fix the path to match your dir structure
output_dir = '../gdrive/MyDrive/EECS598-007_Group/ATTA_output/'
!ls $output_dir

pgd_5_adv_itr1.json  pgd_5_adv_itr1_model.pt  pgd_5_adv_itr1.pt  README.gdoc


In [7]:
transform = transforms.Compose(
    [
      # transforms.Resize((224, 224)),
      transforms.ToTensor(),
      # transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])
cifar_trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
cifar_testset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
cifar_trainloader = torch.utils.data.DataLoader(cifar_trainset, batch_size=64, shuffle=True, num_workers=2)
cifar_testloader = torch.utils.data.DataLoader(cifar_testset, batch_size=64, shuffle=False, num_workers=2)
cifar_classes = [
    'airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck'
]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [8]:
model = ViTForImageClassification.from_pretrained('nateraw/vit-base-patch16-224-cifar10', return_dict=False)
model.to(device)
model = nn.Sequential(
    nn.Upsample(scale_factor=7, mode='bilinear'),
    model
)

class Transformer(nn.Module):
  def __init__(self, model):
    super(Transformer, self).__init__()
    self.model = model
  def forward(self, x):
    return self.model(x)[0]

ViT_model = Transformer(model).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(ViT_model.parameters(), lr=0.01)


# ATTA Train ViT

In [9]:
atk_pgd_5 = torchattacks.PGD(ViT_model, eps=0.05, alpha=2/255, steps=5)
atk_pgd_10 = torchattacks.PGD(ViT_model, eps=0.05, alpha=2/255, steps=10)
atk_pgd_20 = torchattacks.PGD(ViT_model, eps=0.05, alpha=2/255, steps=20)

In [10]:
cifar_adv_test20 = []
start = time.time()
for i, (inputs, labels) in enumerate(cifar_testloader):
    adv_images = atk_pgd_20(inputs, labels)
    cifar_adv_test20.append((adv_images.cpu(), labels.cpu()))
    if i % 50 == 0:
      print("Progress(%): ", i*32/100)

print("total time(sec) : %.2f" % (time.time() - start))

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3458: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)


Progress(%):  0.0
total time(sec) : 7259.35


NameError: ignored

In [11]:
# Store
outfile_name = output_dir + 'pgd_20_adv_test.pt'
!touch $outfile_name
torch.save(cifar_adv_test20, outfile_name)
!ls ../gdrive/MyDrive/EECS598-007_Group/ATTA_output/

pgd_10_adv_test.pt  pgd_5_adv_itr1.json      pgd_5_adv_itr1.pt
pgd_20_adv_test.pt  pgd_5_adv_itr1_model.pt  README.gdoc
